In [1]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 27.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found

In [2]:
#creating new csv 
import pandas as pd

df = pd.read_csv('/kaggle/input/csv-gm8k/gm8k.csv')
df.head()

,question,answer,final_solution
0,Janet’s ducks lay 16 eggs per day. She eats th...,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...,18
1,A robe takes 2 bolts of blue fiber and half th...,It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...,3
2,Josh decides to try flipping a house. He buys...,The cost of the house and repairs came out to ...,70000
3,James decides to run 3 sprints 3 times a week....,He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*...,540
4,"Every day, Wendi feeds each of her chickens th...","If each chicken eats 3 cups of feed per day, t...",20


In [3]:
df.drop('final_solution',axis= 1 , inplace= True)

In [4]:
df.head()
df.to_csv('gm8k_ans.csv', index=False)

In [5]:
'''df = pd.read_csv('G:/Python/fine_tune_improve_mathematical_reasoning/Qwen2.5-1.5B_Fine-tuning/dataset/gm8k_new.csv')
df.head()'''

"df = pd.read_csv('G:/Python/fine_tune_improve_mathematical_reasoning/Qwen2.5-1.5B_Fine-tuning/dataset/gm8k_new.csv')\ndf.head()"

In [6]:
from datasets import Dataset
dataset = Dataset.from_pandas(df[["question", "answer"]])
dataset = dataset.train_test_split(test_size= 0.2)

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import bitsandbytes as bnb  

model_id = "Qwen/Qwen2.5-1.5B"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,           
    device_map="auto",         
    trust_remote_code=True,
)


tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

2025-06-10 18:51:45.010084: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749581505.194863      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749581505.247577      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [8]:
def preprocess_qwen(example):
    full_prompt = f"<|user|>\nQuestion: {example['question']}\n<|assistant|>\n{example['answer']}"
    encoded = tokenizer(full_prompt, truncation=True, padding="max_length", max_length=1024)
    encoded["labels"] = encoded["input_ids"].copy()
    return encoded

tokenized_dataset = dataset.map(preprocess_qwen, batched=False)


Map:   0%|          | 0/1055 [00:00<?, ? examples/s]

Map:   0%|          | 0/264 [00:00<?, ? examples/s]

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

model_id = "Qwen/Qwen2.5-1.5B"


tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)


model = prepare_model_for_kbit_training(model)


lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"], 
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)


model.print_trainable_parameters()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


trainable params: 2,179,072 || all params: 1,545,893,376 || trainable%: 0.1410


In [19]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


training_args = TrainingArguments(
    output_dir="./qwen-lora-finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    fp16=True,  # mixed-precision training
    logging_steps=50,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    report_to="none",  # disable wandb/huggingface logging
    learning_rate=2e-4,
    warmup_steps=100,
    optim="paged_adamw_8bit",  # recommended for 8-bit + LoRA
    label_names=["labels"],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,

)


In [20]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
50,1.188500
100,0.888800
150,0.823300


TrainOutput(global_step=198, training_loss=0.9284252975926255, metrics={'train_runtime': 4118.2378, 'train_samples_per_second': 0.769, 'train_steps_per_second': 0.048, 'total_flos': 2.55229426925568e+16, 'train_loss': 0.9284252975926255, 'epoch': 3.0})

In [25]:
model.save_pretrained("/kaggle/working/qwen-lora-adapters")

In [26]:
tokenizer.save_pretrained("/kaggle/working/qwen-lora-adapters")


('/kaggle/working/qwen-lora-adapters/tokenizer_config.json',
 '/kaggle/working/qwen-lora-adapters/special_tokens_map.json',
 '/kaggle/working/qwen-lora-adapters/vocab.json',
 '/kaggle/working/qwen-lora-adapters/merges.txt',
 '/kaggle/working/qwen-lora-adapters/added_tokens.json',
 '/kaggle/working/qwen-lora-adapters/tokenizer.json')

In [27]:
import os

output_path = "/kaggle/working/qwen-lora-adapters"
print("Exists:", os.path.exists(output_path))
print("Contents:", os.listdir(output_path) if os.path.exists(output_path) else "Not found")


Exists: True
Contents: ['added_tokens.json', 'special_tokens_map.json', 'tokenizer_config.json', 'README.md', 'tokenizer.json', 'adapter_model.safetensors', 'adapter_config.json', 'merges.txt', 'vocab.json']


In [28]:
import shutil
shutil.make_archive("/kaggle/working/qwen-lora-adapters", 'zip', "/kaggle/working/qwen-lora-adapters")


'/kaggle/working/qwen-lora-adapters.zip'